https://github.com/lineality/CS-Build-Week-1

https://github.com/CS27-spring-break-bw-megateam/CS-Build-Week-1/blob/staging-heroku/util/sample_generator.py

Are linked rooms like linked notes in a linked-list or graph?
(or a hash-table?)

Q: Where are the rooms stored?


To do: 
Export model of maze?

https://stackoverflow.com/questions/38502/whats-a-good-algorithm-to-generate-a-maze

https://github.com/theJollySin/mazelib


# original create_world.py put into shell


In [0]:

# create_world.py
# this is the file that must be run, which dungeon making must be merged into
from django.contrib.auth.models import User
from adventure.models import Player, Room
# steps:
# step 1: make rooms
# step 2: make players
#
# erases pre-existing rooms
Room.objects.all().delete()
# creates set rooms
# randomly generated/(or selected-from-smaller-set) description
# idea: create clusters of rooms, and connect those
r_outside = Room(title="Outside Cave Entrance", description="North of you, the cave mount beckons")
r_foyer = Room(title="Foyer", description="""Dim light filters in from the south. Dusty passages run north and east.""")
r_overlook = Room(title="Grand Overlook", description="""A steep cliff appears before you, falling into the darkness. Ahead to the north, a light flickers in the distance, but there is no way across the chasm.""")
r_narrow = Room(title="Narrow Passage", description="""The narrow passage bends here from west to north. The smell of gold permeates the air.""")
r_treasure = Room(title="Treasure Chamber", description="""You've found the long-lost treasure chamber! Sadly, it has already been completely emptied by earlier adventurers. The only exit is to the south.""")
# .save() saves in the django database
r_outside.save()
r_foyer.save()
r_overlook.save()
r_narrow.save()
r_treasure.save()
# Link rooms together
r_outside.connectRooms(r_foyer, "n")
r_foyer.connectRooms(r_outside, "s")
r_foyer.connectRooms(r_overlook, "n")
r_overlook.connectRooms(r_foyer, "s")
r_foyer.connectRooms(r_narrow, "e")
r_narrow.connectRooms(r_foyer, "w")
r_narrow.connectRooms(r_treasure, "n")
r_treasure.connectRooms(r_narrow, "s")
players = Player.objects.all()
for p in players:
    p.currentRoom = r_outside.id
    p.save()

## idea. start with a grid of rooms: 100x100
make 100 random moves through that grid of rooms.
each time, pick one of 5-10 room types with similar descriptions
and connect to the past room (using the same function)




In [0]:
this_room_type[1]

'The wooden room is very...woodish.'

# Room Types

In [0]:
            # standard room types
            r_long_hallway = ("Long Hallway", "This hallway is very long. Be careful.")
            r_short_hallway = ("Short Hallway", "This hallway is not very long. Be very careful.")
            r_medium_hallway = ("Medium Hallway", "This hallway is... Be very careful.")
            r_stone_room = ("stone room", "This stone room is dark. Be careful.")
            r_wooden_room = ("wooden_room", "The wooden room is very...woodish.")
            r_swimming_pool = ("Swimming Pool", "This looks like a great place to swim...is that a giant squid?")
            r_library = ("library", "This library is very bright and quiet.")
            r_storage_room = ("storage_room", "This storage room is full of things.")
            r_kitchen = ("Kitchen", "This room is full of cooking supplies.")
            r_dungeon_cell = ("Dungeon Cell", "This room looks like a prison cell.")
            r_server_room = ("Dungeon Server Wearhouse", "This where the monsters keep their servers.")
            r_banquet_room = ("Banquet Room", "This looks like a room for large events.")
            r_nest = ("Nest", "This looks web filled nest for some kind of creature.")
            r_armory = ("Armory", "This stone room is full of arms. To Arms!")
            r_cave = ("Cave", "This stone room looks to be a natural cave opening.")
            r_mine = ("Mine", "This area looks to be mine set up to find coal, iron, diamonds...and redstone...Punch a tree and get your torches ready!")
            r_office = ("Office", "This room looks like a basement office at the state department.")           
            r_walk_in_closet = ("Walk In Closet", "This room is a huge walk-in-closet. There's clothing everywhere! What????")
            # List of possible room types of Rooms
            room_type_list = [r_long_hallway, r_medium_hallway, r_short_hallway, r_stone_room, r_wooden_room, r_swimming_pool, r_library, r_storage_room, r_kitchen, r_dungeon_cell, r_server_room, r_banquet_room, r_nest, r_armory, r_mine, r_office, r_walk_in_closet]

In [17]:
import random
this_room_type = random.choice(room_type_list)
print(this_room_type[0])
print(this_room_type[1])

library
This library is very bright and quiet.


# Modified Sample_generator for making random rooms

In [29]:
# merging code
# known bug: may loop if dungeon-grid size is too small

# 2 lists: generated as the rooms are made
# list of rooms  [{x: 4, y: 5},{x: 4, y: 5}, ...]
# list of connections  [ [{x: 4, y: 5}, {x: 4, y: 5}], 
#                        [{x: 4, y: 5}, {x: 4, y: 5}] ]

class Room_maker:  # Class: Room input parameters (id, name, description, x, y)
    # constructor
    def __init__(self, id, name, description, x, y):
        # attributes
        self.id = id
        self.name = name
        self.description = description
        self.n_to = None
        self.s_to = None
        self.e_to = None
        self.w_to = None
        self.x = x
        self.y = y

    # ?
    # class methods
    def __repr__(self):
        # if self.e_to is not None:
        #     return f"({self.x}, {self.y}) -> ({self.e_to.x}, {self.e_to.y})"
        return f"({self.x}, {self.y})"

    # connects to the previously make room
    def connect_rooms(self, connecting_room, direction):
        # Connect two rooms in the given n/s/e/w direction
        reverse_dirs = {"n": "s", "s": "n", "e": "w", "w": "e"}
        reverse_dir = reverse_dirs[direction]
        setattr(self, f"{direction}_to", connecting_room)
        setattr(connecting_room, f"{reverse_dir}_to", self)

    # this is just for the player
    def get_room_in_direction(self, direction):
        return getattr(self, f"{direction}_to")


# see while loop for making random rooms
class World:  # Class
    # constructor
    def __init__(self):
        # attributes of world-grid
        self.grid = None
        # # 2 lists for front end mapping
        self.rooms_list = None
        self.doors_list = None
        self.width = 0
        self.height = 0

    # class methods    
    def generate_rooms(self, size_x, size_y, num_rooms):
        # Step 1: set up variables and things for while loop
        # Step 2: run while loop to create rooms

        # this makes any empty world grid - why this sequence?
        #
        # make part of grid? sets y dimension?
        self.grid = [None] * size_y
        # sets max size of world-grid
        self.width = size_x        
        self.height = size_y
        # ? since grid was made, what does this do?
        # sets x dimension?
        for i in range( len(self.grid) ):
            self.grid[i] = [None] * size_x
        #
        # # 2 lists for front end mapping
        self.rooms_list = []
        self.doors_list = []

        # Start from lower-left corner (0,0)
        x = -1 # (this will become 0 on the first step)
        y = 0

        # keeping values for comparison
        previous_id_tuple = None
        previous_room = None
        previous_direction = None

        # up to max number of rooms
        room_count = 0

        # move these out of the while loop
        import random
        # standard room types
        r_long_hallway = ("Long Hallway", "This hallway is very long. Be careful.")
        r_short_hallway = ("Short Hallway", "This hallway is not very long. Be very careful.")
        r_medium_hallway = ("Medium Hallway", "This hallway is... Be very careful.")
        r_stone_room = ("stone room", "This stone room is dark. Be careful.")
        r_wooden_room = ("wooden_room", "The wooden room is very...woodish.")
        r_swimming_pool = ("Swimming Pool", "This looks like a great place to swim...is that a giant squid?")
        r_library = ("library", "This library is very bright and quiet.")
        r_storage_room = ("storage_room", "This storage room is full of things.")
        r_kitchen = ("Kitchen", "This room is full of cooking supplies.")
        r_dungeon_cell = ("Dungeon Cell", "This room looks like a prison cell.")
        r_server_room = ("Dungeon Server Wearhouse", "This where the monsters keep their servers.")
        r_banquet_room = ("Banquet Room", "This looks like a room for large events.")
        r_nest = ("Nest", "This looks web filled nest for some kind of creature.")
        r_armory = ("Armory", "This stone room is full of arms. To Arms!")
        r_cave = ("Cave", "This stone room looks to be a natural cave opening.")
        r_mine = ("Mine", "This area looks to be mine set up to find coal, iron, diamonds...and redstone...Punch a tree and get your torches ready!")
        r_office = ("Office", "This room looks like a basement office at the state department.")           
        r_walk_in_closet = ("Walk In Closet", "This room is a huge walk-in-closet. There's clothing everywhere!")

        # List of possible room types of Rooms
        room_type_list = [r_long_hallway, r_medium_hallway, r_short_hallway, r_stone_room, r_wooden_room, r_swimming_pool, r_library, r_storage_room, r_kitchen, r_dungeon_cell, r_server_room, r_banquet_room, r_nest, r_armory, r_mine, r_office, r_walk_in_closet]

        # set values
        x = 0
        y = 0

        current_direction = 'n'

        # while look for making rooms
        # Random walk through grid of rooms, 
        # any direction except backwards
        while room_count < num_rooms:

            # randomly pick room type
            this_room_type = random.choice(room_type_list)

            # # inspection
            # print(this_room_type)

            # previous_room_direction = reverse_dirs[current_direction]
            possible_directions = ['n', 's', 'e' ,'w']
            # starts with a random choice:
            current_direction = random.choice(possible_directions)

            # save old values
            x_old = x
            y_old = y

            # save the location of the past room
            past_room = (x,y)

            # "don't go out of bounds"
            # check to make sure not out of bounds
            # while out of bounds, keep trying again for a not-out option
            while (x >= size_x - 1) or (y >= size_y - 1) or (x < 0) or (y < 0) or ((x,y) == past_room) or ((x,y) == (0,0)):
                # save old values
                x = x_old
                y = y_old

                # starts with a random choice:
                current_direction = random.choice(possible_directions)
                if current_direction == 'n':
                    y += 1
                if current_direction == 's':
                    y -= 1
                if current_direction == 'e':
                    x += 1
                if current_direction == 'w':
                    x -= 1
            
            # # inspection
            # print("room picked", x,y)

            # look for a previously made room (where you are looking 
            # to make a new room)
            # if not (already one there), make a new room
            if self.grid[y][x] is None:
                # Create a room in the given direction
                room = Room_maker(room_count, this_room_type[0], this_room_type[1], x, y)
              # room = Room(room_count, this_room_type[0], this_room_type[1])
              # room = Room(room_count, this_room_type[0], this_room_type[1], x, y)   

                # Save the room in the World grid
                self.grid[y][x] = room
                # update room counter, only if a new room is made
                room_count += 1
                # add room to room list:
                mask_dict = {'x':x, 'y':y}
                self.rooms_list.extend([mask_dict])

                # # inspection
                # print("mask dict", mask_dict)
                # print("x,y", x,y)

            # sets previous reverse room direction
            reverse_dirs = {"n": "s", "s": "n", "e": "w", "w": "e"}
            previous_room_direction = reverse_dirs[current_direction]

            # either way: Connect the new room to the previous room
            if previous_room is not None:
                # connect to the previous room...in DB
                previous_room.connect_rooms(room, previous_room_direction)
                room.connect_rooms(previous_room, current_direction)
                # 
                # connect to the previous room...in room maker


                # doors list: add room to list:
                mask = [{'x': int(room.x), 'y': int(room.y)}, {'x': int(previous_room.x), 'y': int(previous_room.y)}]
                self.doors_list.extend([mask])

                # # inspection
                # print(mask)

            # Update iteration variables
            previous_room = room

            # inspection
            # print("room count", room_count)

            # # save room to django datatable
            # room.save()

# # generate world:
# # set input values
w = World()
num_rooms = 100
width = 20
height = 20
w.generate_rooms(width, height, num_rooms)
#
# inspection
print(w.rooms_list)
print(w.doors_list)
# Where will these files be stored for later extraction?

[{'x': 0, 'y': 1}, {'x': 0, 'y': 2}, {'x': 0, 'y': 3}, {'x': 1, 'y': 1}, {'x': 2, 'y': 1}, {'x': 2, 'y': 0}, {'x': 3, 'y': 0}, {'x': 1, 'y': 0}, {'x': 3, 'y': 1}, {'x': 3, 'y': 2}, {'x': 4, 'y': 0}, {'x': 4, 'y': 1}, {'x': 4, 'y': 2}, {'x': 4, 'y': 3}, {'x': 5, 'y': 3}, {'x': 6, 'y': 3}, {'x': 6, 'y': 4}, {'x': 5, 'y': 4}, {'x': 5, 'y': 5}, {'x': 4, 'y': 5}, {'x': 5, 'y': 6}, {'x': 5, 'y': 7}, {'x': 6, 'y': 7}, {'x': 5, 'y': 8}, {'x': 4, 'y': 8}, {'x': 3, 'y': 8}, {'x': 6, 'y': 8}, {'x': 6, 'y': 9}, {'x': 5, 'y': 9}, {'x': 5, 'y': 10}, {'x': 4, 'y': 10}, {'x': 4, 'y': 9}, {'x': 3, 'y': 9}, {'x': 7, 'y': 8}, {'x': 7, 'y': 9}, {'x': 7, 'y': 10}, {'x': 7, 'y': 11}, {'x': 6, 'y': 10}, {'x': 5, 'y': 11}, {'x': 4, 'y': 11}, {'x': 5, 'y': 12}, {'x': 5, 'y': 13}, {'x': 5, 'y': 14}, {'x': 5, 'y': 15}, {'x': 6, 'y': 15}, {'x': 6, 'y': 14}, {'x': 6, 'y': 13}, {'x': 4, 'y': 14}, {'x': 4, 'y': 13}, {'x': 3, 'y': 13}, {'x': 3, 'y': 14}, {'x': 2, 'y': 14}, {'x': 1, 'y': 14}, {'x': 1, 'y': 13}, {'x': 

In [0]:
dictionary_of_rooms_and_doors = {(0,0):1}
dictionary_of_rooms_and_doors[(0,0)] = [3]
dictionary_of_rooms_and_doors
type(dictionary_of_rooms_and_doors[(0,0)])

list

In [0]:
previous_id_tuple = (0,1)
dictionary_of_rooms_and_doors = {(0,0): []}
#dictionary_of_rooms_and_doors = {(0,0)}
dictionary_of_rooms_and_doors
dictionary_of_rooms_and_doors[(0,0)].extend(previous_id_tuple)
dictionary_of_rooms_and_doors


{(0, 0): [0, 1]}

merge the coconut_generator and the create_world.py

In [0]:

# create_world.py
# this is the file that must be run, which dungeon making must be merged into
from django.contrib.auth.models import User
from adventure.models import Player, Room
# steps:
# step 1: make rooms
# step 2: make players
#
# erases pre-existing rooms
Room.objects.all().delete()
# creates set rooms
# randomly generated/(or selected-from-smaller-set) description
# idea: create clusters of rooms, and connect those
r_outside = Room(title="Outside Cave Entrance", description="North of you, the cave mount beckons")
r_foyer = Room(title="Foyer", description="""Dim light filters in from the south. Dusty passages run north and east.""")
r_overlook = Room(title="Grand Overlook", description="""A steep cliff appears before you, falling into the darkness. Ahead to the north, a light flickers in the distance, but there is no way across the chasm.""")
r_narrow = Room(title="Narrow Passage", description="""The narrow passage bends here from west to north. The smell of gold permeates the air.""")
r_treasure = Room(title="Treasure Chamber", description="""You've found the long-lost treasure chamber! Sadly, it has already been completely emptied by earlier adventurers. The only exit is to the south.""")
#
# .save() saves in the django database
r_outside.save()
r_foyer.save()
r_overlook.save()
r_narrow.save()
r_treasure.save()
# 
# Link rooms together
r_outside.connectRooms(r_foyer, "n")
r_foyer.connectRooms(r_outside, "s")
r_foyer.connectRooms(r_overlook, "n")
r_overlook.connectRooms(r_foyer, "s")
r_foyer.connectRooms(r_narrow, "e")
r_narrow.connectRooms(r_foyer, "w")
r_narrow.connectRooms(r_treasure, "n")
r_treasure.connectRooms(r_narrow, "s")
#
# make players
players = Player.objects.all()
for p in players:
    p.currentRoom = r_outside.id
    p.save()

array of rooms 

grid of rooms and links by coordinates

separate grids to request

2d grid
grid fields:
  - field: room or not (1,0)

(maybe this is all that's needed)
dictionary of doors:
  - key = that room (coordinates)
  - values = coordinates to rooms connected to


move endpoint:
  - give you the room description etc
  - to do: send coordinates


# new plan!

2 lists, not one

## 1. list of connections = []

list of dictionies:
each list contains 2 dictionaries
each dictionary contains 2 key value pairs
which are the x y coordinary

## 2. list of rooms


? Add new classes into the code, e.g. to build the world?
functions, classes, vs. models?(DB)


If the coordinates are not needed, then we could separate the world-maker class from the world-storage class e.g.
keep the same Room() class, but also have Room_maker()

# model.py

In [0]:

# create_world.py
# this is the file that must be run, which dungeon making must be merged into
from django.contrib.auth.models import User
from adventure.models import Player, Room
# steps:
# step 1: make rooms
# step 2: make players
#
# erases pre-existing rooms
Room.objects.all().delete()
# creates set rooms
# randomly generated/(or selected-from-smaller-set) description
# idea: create clusters of rooms, and connect those
r_outside = Room(title="Outside Cave Entrance", description="North of you, the cave mount beckons")
r_foyer = Room(title="Foyer", description="""Dim light filters in from the south. Dusty passages run north and east.""")
r_overlook = Room(title="Grand Overlook", description="""A steep cliff appears before you, falling into the darkness. Ahead to the north, a light flickers in the distance, but there is no way across the chasm.""")
r_narrow = Room(title="Narrow Passage", description="""The narrow passage bends here from west to north. The smell of gold permeates the air.""")
r_treasure = Room(title="Treasure Chamber", description="""You've found the long-lost treasure chamber! Sadly, it has already been completely emptied by earlier adventurers. The only exit is to the south.""")
#
# .save() saves in the django database
r_outside.save()
r_foyer.save()
r_overlook.save()
r_narrow.save()
r_treasure.save()
# 
# Link rooms together
r_outside.connectRooms(r_foyer, "n")
r_foyer.connectRooms(r_outside, "s")
r_foyer.connectRooms(r_overlook, "n")
r_overlook.connectRooms(r_foyer, "s")
r_foyer.connectRooms(r_narrow, "e")
r_narrow.connectRooms(r_foyer, "w")
r_narrow.connectRooms(r_treasure, "n")
r_treasure.connectRooms(r_narrow, "s")
#
# make players
players = Player.objects.all()
for p in players:
    p.currentRoom = r_outside.id
    p.save()

It looks like the main goals for today (this morning) for backend are:
- 1. connect the (apparently working) room generator to the create_world.py file
- 2. add x,y properties to the rooms in the database
- 3. add room x,y to the output of /move/ endpoint
- 4. make sure the rooms_list and doors_list are accessible for front-end to ask for them. (maybe add a 'world' model in which these are stored?) 

In [0]:
# merging code
# known bug: may loop if dungeon-grid size is too small
# 2 lists: generated as the rooms are made
# list of rooms  [{x: 4, y: 5},{x: 4, y: 5}, ...]
# list of connections  [ [{x: 4, y: 5}, {x: 4, y: 5}], 
#                        [{x: 4, y: 5}, {x: 4, y: 5}] ]
class Room_maker:  # Class: Room input parameters (id, name, description, x, y)
    # constructor
    def __init__(self, id, name, description, x, y):
        # attributes
        self.id = id
        self.name = name
        self.description = description
        self.n_to = None
        self.s_to = None
        self.e_to = None
        self.w_to = None
        self.x = x
        self.y = y
    # ?
    # class methods
    def __repr__(self):
        # if self.e_to is not None:
        #     return f"({self.x}, {self.y}) -> ({self.e_to.x}, {self.e_to.y})"
        return f"({self.x}, {self.y})"
    # connects to the previously make room
    def maker_connect_rooms(self, connecting_room, direction):
        # Connect two rooms in the given n/s/e/w direction
        reverse_dirs = {"n": "s", "s": "n", "e": "w", "w": "e"}
        reverse_dir = reverse_dirs[direction]
        setattr(self, f"{direction}_to", connecting_room)
        setattr(connecting_room, f"{reverse_dir}_to", self)
    # this is just for the player
    def get_room_in_direction(self, direction):
        return getattr(self, f"{direction}_to")
# see while loop for making random rooms
class World:  # Class
    # constructor
    def __init__(self):
        # attributes of world-grid
        self.grid = None
        # # 2 lists for front end mapping
        self.rooms_list = None
        self.doors_list = None
        self.width = 0
        self.height = 0
    # class methods    
    def generate_rooms(self, size_x, size_y, num_rooms):
        # Step 1: set up variables and things for while loop
        # Step 2: run while loop to create rooms
        # this makes any empty world grid - why this sequence?
        #
        # make part of grid? sets y dimension?
        self.grid = [None] * size_y
        # sets max size of world-grid
        self.width = size_x        
        self.height = size_y
        # ? since grid was made, what does this do?
        # sets x dimension?
        for i in range( len(self.grid) ):
            self.grid[i] = [None] * size_x
        #
        # # 2 lists for front end mapping
        self.rooms_list = []
        self.doors_list = []
        # Start from lower-left corner (0,0)
        x = -1 # (this will become 0 on the first step)
        y = 0
        # keeping values for comparison
        previous_id_tuple = None
        previous_room = None
        previous_direction = None
        # up to max number of rooms
        room_count = 0
        # move these out of the while loop
        import random
        # standard room types
        r_long_hallway = ("Long Hallway", "This hallway is very long. Be careful.")
        r_short_hallway = ("Short Hallway", "This hallway is not very long. Be very careful.")
        r_medium_hallway = ("Medium Hallway", "This hallway is... Be very careful.")
        r_stone_room = ("Stone Room", "This stone room is dark. Be careful.")
        r_wooden_room = ("Wooden Room", "The wooden room is very...woodish.")
        r_swimming_pool = ("Swimming Pool", "This looks like a great place to swim...is that a giant squid?")
        r_library = ("library", "This library is very bright and quiet.")
        r_storage_room = ("storage_room", "This storage room is full of things.")
        r_kitchen = ("Kitchen", "This room is full of cooking supplies.")
        r_dungeon_cell = ("Dungeon Cell", "This room looks like a prison cell.")
        r_server_room = ("Dungeon Server Wearhouse", "This where the monsters keep their servers.")
        r_banquet_room = ("Banquet Room", "This looks like a room for large events.")
        r_nest = ("Nest", "This looks web filled nest for some kind of creature.")
        r_armory = ("Armory", "This stone room is full of arms. To Arms!")
        r_cave = ("Cave", "This stone room looks to be a natural cave opening.")
        r_mine = ("Mine", "This area looks to be mine set up to find coal, iron, diamonds...and redstone...Punch a tree and get your torches ready!")
        r_office = ("Office", "This room looks like a basement office at the state department.")           
        r_walk_in_closet = ("Walk In Closet", "This room is a huge walk-in-closet. There's clothing everywhere! What????")
        r_ball_room = ("Ball Room", "This room is full to waist height of small plastic multi-colored balls...be careful!!!")
        # List of possible room types of Rooms
        room_type_list = [r_long_hallway, r_medium_hallway, r_short_hallway, r_stone_room, r_wooden_room, r_swimming_pool, r_library, r_storage_room, r_kitchen, r_dungeon_cell, r_server_room, r_banquet_room, r_nest, r_armory, r_mine, r_office, r_walk_in_closet, r_ball_room]
        # set values
        x = 0
        y = 0
        current_direction = 'n'
        # while look for making rooms
        # Random walk through grid of rooms, 
        # any direction except backwards
        while room_count < num_rooms:
            # randomly pick room type
            this_room_type = random.choice(room_type_list)
            # # inspection
            # print(this_room_type)
            # previous_room_direction = reverse_dirs[current_direction]
            possible_directions = ['n', 's', 'e' ,'w']
            # starts with a random choice:
            current_direction = random.choice(possible_directions)
            # save old values
            x_old = x
            y_old = y
            # save the location of the past room
            past_room = (x,y)
            # "don't go out of bounds"
            # check to make sure not out of bounds
            # while out of bounds, keep trying again for a not-out option
            while (x >= size_x - 1) or (y >= size_y - 1) or (x < 0) or (y < 0) or ((x,y) == past_room) or ((x,y) == (0,0)):
                # save old values
                x = x_old
                y = y_old
                # starts with a random choice:
                current_direction = random.choice(possible_directions)
                if current_direction == 'n':
                    y += 1
                if current_direction == 's':
                    y -= 1
                if current_direction == 'e':
                    x += 1
                if current_direction == 'w':
                    x -= 1
            # # inspection
            # print("room picked", x,y)
            # look for a previously made room (where you are looking 
            # to make a new room)
            # if not (already one there), make a new room
            if self.grid[y][x] is None:
                # Create a room in the given direction
                maker_room = Room_maker(room_count, this_room_type[0], this_room_type[1], x, y)
                # room = Room(room_count, this_room_type[0], this_room_type[1])
                room = Room(room_count, this_room_type[0], this_room_type[1], x, y)   
                # Save the room in the World grid
                self.grid[y][x] = maker_room
                # update room counter, only if a new room is made
                room_count += 1
                # add room to room list:
                mask_dict = {'x':x, 'y':y}
                self.rooms_list.extend([mask_dict])
            # sets previous reverse room direction
            reverse_dirs = {"n": "s", "s": "n", "e": "w", "w": "e"}
            previous_room_direction = reverse_dirs[current_direction]
            # either way: Connect the new room to the previous room
            if previous_room is not None:
                # connect to the previous room...in DB
                previous_room.maker_connect_rooms(maker_room, previous_room_direction)
                maker_room.maker_connect_rooms(previous_room, current_direction)
                # # connect to the previous room...in room maker
                previous_room.connect_rooms(room, previous_room_direction)
                room.connect_rooms(previous_room, current_direction)
                # doors list: add room to list:
                mask = [{'x': int(maker_room.x), 'y': int(maker_room.y)}, {'x': int(previous_room.x), 'y': int(previous_room.y)}]
                self.doors_list.extend([mask])
            # Update iteration variables
            previous_room = maker_room
            # Update iteration variables
            previous_room = maker_room
            # # save room to django datatable
            # room.save()
# # generate world:
# # set input values
w = World()
num_rooms = 100
width = 20
height = 20
w.generate_rooms(width, height, num_rooms)
#
# inspection
# print(w.rooms_list)
# print(w.doors_list)
# Where will these files be stored for later extraction?

# hybrid code


In [0]:
# merging code
# known bug: may loop if dungeon-grid size is too small
# 2 lists: generated as the rooms are made
# list of rooms  [{x: 4, y: 5},{x: 4, y: 5}, ...]
# list of connections  [ [{x: 4, y: 5}, {x: 4, y: 5}], 
#                        [{x: 4, y: 5}, {x: 4, y: 5}] ]
class Room_maker:  # Class: Room input parameters (id, name, description, x, y)
    # constructor
    def __init__(self, id, name, description, x, y):
        # attributes
        self.id = id
        self.name = name
        self.description = description
        self.n_to = None
        self.s_to = None
        self.e_to = None
        self.w_to = None
        self.x = x
        self.y = y
    # ?
    # class methods
    def __repr__(self):
        # if self.e_to is not None:
        #     return f"({self.x}, {self.y}) -> ({self.e_to.x}, {self.e_to.y})"
        return f"({self.x}, {self.y})"
    # connects to the previously make room
    def maker_connect_rooms(self, connecting_room, direction):
        # Connect two rooms in the given n/s/e/w direction
        reverse_dirs = {"n": "s", "s": "n", "e": "w", "w": "e"}
        reverse_dir = reverse_dirs[direction]
        setattr(self, f"{direction}_to", connecting_room)
        setattr(connecting_room, f"{reverse_dir}_to", self)
    # this is just for the player
    def get_room_in_direction(self, direction):
        return getattr(self, f"{direction}_to")
# see while loop for making random rooms
class World:  # Class
    # constructor
    def __init__(self):
        # attributes of world-grid
        self.grid = None
        # # 2 lists for front end mapping
        self.rooms_list = None
        self.doors_list = None
        self.width = 0
        self.height = 0
    # class methods    
    def generate_rooms(self, size_x, size_y, num_rooms):
        # Step 1: set up variables and things for while loop
        # Step 2: run while loop to create rooms
        # this makes any empty world grid - why this sequence?
        #
        # make part of grid? sets y dimension?
        self.grid = [None] * size_y
        # sets max size of world-grid
        self.width = size_x        
        self.height = size_y
        # ? since grid was made, what does this do?
        # sets x dimension?
        for i in range( len(self.grid) ):
            self.grid[i] = [None] * size_x
        #
        # # 2 lists for front end mapping
        self.rooms_list = []
        self.doors_list = []
        # Start from lower-left corner (0,0)
        x = -1 # (this will become 0 on the first step)
        y = 0
        # keeping values for comparison
        previous_id_tuple = None
        previous_room = None
        previous_direction = None
        # up to max number of rooms
        room_count = 0
        # move these out of the while loop
        import random
        # standard room types
        r_long_hallway = ("Long Hallway", "This hallway is very long. Be careful.")
        r_short_hallway = ("Short Hallway", "This hallway is not very long. Be very careful.")
        r_medium_hallway = ("Medium Hallway", "This hallway is... Be very careful.")
        r_stone_room = ("Stone Room", "This stone room is dark. Be careful.")
        r_wooden_room = ("Wooden Room", "The wooden room is very...woodish.")
        r_swimming_pool = ("Swimming Pool", "This looks like a great place to swim...is that a giant squid?")
        r_library = ("library", "This library is very bright and quiet.")
        r_storage_room = ("storage_room", "This storage room is full of things.")
        r_kitchen = ("Kitchen", "This room is full of cooking supplies.")
        r_dungeon_cell = ("Dungeon Cell", "This room looks like a prison cell.")
        r_server_room = ("Dungeon Server Wearhouse", "This where the monsters keep their servers.")
        r_banquet_room = ("Banquet Room", "This looks like a room for large events.")
        r_nest = ("Nest", "This looks web filled nest for some kind of creature.")
        r_armory = ("Armory", "This stone room is full of arms. To Arms!")
        r_cave = ("Cave", "This stone room looks to be a natural cave opening.")
        r_mine = ("Mine", "This area looks to be mine set up to find coal, iron, diamonds...and redstone...Punch a tree and get your torches ready!")
        r_office = ("Office", "This room looks like a basement office at the state department.")           
        r_walk_in_closet = ("Walk In Closet", "This room is a huge walk-in-closet. There's clothing everywhere! What????")
        r_ball_room = ("Ball Room", "This room is full to waist height of small plastic multi-colored balls...be careful!!!")
        # List of possible room types of Rooms
        room_type_list = [r_long_hallway, r_medium_hallway, r_short_hallway, r_stone_room, r_wooden_room, r_swimming_pool, r_library, r_storage_room, r_kitchen, r_dungeon_cell, r_server_room, r_banquet_room, r_nest, r_armory, r_mine, r_office, r_walk_in_closet, r_ball_room]
        # set values
        x = 0
        y = 0
        current_direction = 'n'
        # while look for making rooms
        # Random walk through grid of rooms, 
        # any direction except backwards
        while room_count < num_rooms:
            # randomly pick room type
            this_room_type = random.choice(room_type_list)
            # # inspection
            # print(this_room_type)
            # previous_room_direction = reverse_dirs[current_direction]
            possible_directions = ['n', 's', 'e' ,'w']
            # starts with a random choice:
            current_direction = random.choice(possible_directions)
            # save old values
            x_old = x
            y_old = y
            # save the location of the past room
            past_room = (x,y)
            # "don't go out of bounds"
            # check to make sure not out of bounds
            # while out of bounds, keep trying again for a not-out option
            while (x >= size_x - 1) or (y >= size_y - 1) or (x < 0) or (y < 0) or ((x,y) == past_room) or ((x,y) == (0,0)):
                # save old values
                x = x_old
                y = y_old
                # starts with a random choice:
                current_direction = random.choice(possible_directions)
                if current_direction == 'n':
                    y += 1
                if current_direction == 's':
                    y -= 1
                if current_direction == 'e':
                    x += 1
                if current_direction == 'w':
                    x -= 1
            # # inspection
            # print("room picked", x,y)
            # look for a previously made room (where you are looking 
            # to make a new room)
            # if not (already one there), make a new room
            if self.grid[y][x] is None:
                # Create a room in the given direction
                maker_room = Room_maker(room_count, this_room_type[0], this_room_type[1], x, y)
                # room = Room(room_count, this_room_type[0], this_room_type[1])
                room = Room(room_count, this_room_type[0], this_room_type[1], x, y)   
                # Save the room in the World grid
                self.grid[y][x] = maker_room
                # update room counter, only if a new room is made
                room_count += 1
                # add room to room list:
                mask_dict = {'x':x, 'y':y}
                self.rooms_list.extend([mask_dict])
            # sets previous reverse room direction
            reverse_dirs = {"n": "s", "s": "n", "e": "w", "w": "e"}
            previous_room_direction = reverse_dirs[current_direction]
            # either way: Connect the new room to the previous room
            if previous_room is not None:
                # connect to the previous room...in DB
                previous_room.maker_connect_rooms(maker_room, previous_room_direction)
                maker_room.maker_connect_rooms(previous_room, current_direction)
                # # connect to the previous room...in room maker
                previous_room.connect_rooms(room, previous_room_direction)
                room.connect_rooms(previous_room, current_direction)
                # doors list: add room to list:
                mask = [{'x': int(maker_room.x), 'y': int(maker_room.y)}, {'x': int(previous_room.x), 'y': int(previous_room.y)}]
                self.doors_list.extend([mask])
            # Update iteration variables
            previous_room = maker_room
            # Update iteration variables
            previous_room = maker_room
            # # save room to django datatable
            #room.save()
# # generate world:
# # set input values
w = World()
num_rooms = 100
width = 20
height = 20
w.generate_rooms(width, height, num_rooms)

In [0]:
# create_world.py / this version only works in heroku
# this is the file that must be run, which dungeon making must be merged into
from django.contrib.auth.models import User
from adventure.models import Player, Room
# steps:
# step 1: make rooms
# step 2: make players
#
# erases pre-existing rooms
Room.objects.all().delete()
# creates set rooms
# randomly generated/(or selected-from-smaller-set) description
# merging code
# known bug: may loop if dungeon-grid size is too small
# 2 lists: generated as the rooms are made
# list of rooms  [{x: 4, y: 5},{x: 4, y: 5}, ...]
# list of connections  [ [{x: 4, y: 5}, {x: 4, y: 5}], 
#                        [{x: 4, y: 5}, {x: 4, y: 5}] ]
class Room_maker:  # Class: Room input parameters (id, name, description, x, y)
    # constructor
    def __init__(self, id, name, description, x, y):
        # attributes
        self.id = id
        self.name = name
        self.description = description
        self.n_to = None
        self.s_to = None
        self.e_to = None
        self.w_to = None
        self.x = x
        self.y = y
    # ?
    # class methods
    def __repr__(self):
        # if self.e_to is not None:
        #     return f"({self.x}, {self.y}) -> ({self.e_to.x}, {self.e_to.y})"
        return f"({self.x}, {self.y})"
    # connects to the previously make room
    def maker_connect_rooms(self, connecting_room, direction):
        # Connect two rooms in the given n/s/e/w direction
        reverse_dirs = {"n": "s", "s": "n", "e": "w", "w": "e"}
        reverse_dir = reverse_dirs[direction]
        setattr(self, f"{direction}_to", connecting_room)
        setattr(connecting_room, f"{reverse_dir}_to", self)
    # this is just for the player
    def get_room_in_direction(self, direction):
        return getattr(self, f"{direction}_to")
# see while loop for making random rooms
class World:  # Class
    # constructor
    def __init__(self):
        # attributes of world-grid
        self.grid = None
        # # 2 lists for front end mapping
        self.rooms_list = None
        self.doors_list = None
        self.width = 0
        self.height = 0
    # class methods    
    def generate_rooms(self, size_x, size_y, num_rooms):
        # Step 1: set up variables and things for while loop
        # Step 2: run while loop to create rooms
        # this makes any empty world grid - why this sequence?
        #
        # make part of grid? sets y dimension?
        self.grid = [None] * size_y
        # sets max size of world-grid
        self.width = size_x        
        self.height = size_y
        # ? since grid was made, what does this do?
        # sets x dimension?
        for i in range( len(self.grid) ):
            self.grid[i] = [None] * size_x
        #
        # # 2 lists for front end mapping
        self.rooms_list = []
        self.doors_list = []
        # Start from lower-left corner (0,0)
        x = -1 # (this will become 0 on the first step)
        y = 0
        # keeping values for comparison
        previous_id_tuple = None
        previous_room = None
        previous_direction = None
        # up to max number of rooms
        room_count = 0
        # move these out of the while loop
        import random
        # standard room types
        r_long_hallway = ("Long Hallway", "This hallway is very long. Be careful.")
        r_short_hallway = ("Short Hallway", "This hallway is not very long. Be very careful.")
        r_medium_hallway = ("Medium Hallway", "This hallway is... Be very careful.")
        r_stone_room = ("Stone Room", "This stone room is dark. Be careful.")
        r_wooden_room = ("Wooden Room", "The wooden room is very...woodish.")
        r_swimming_pool = ("Swimming Pool", "This looks like a great place to swim...is that a giant squid?")
        r_library = ("library", "This library is very bright and quiet.")
        r_storage_room = ("storage_room", "This storage room is full of things.")
        r_kitchen = ("Kitchen", "This room is full of cooking supplies.")
        r_dungeon_cell = ("Dungeon Cell", "This room looks like a prison cell.")
        r_server_room = ("Dungeon Server Wearhouse", "This where the monsters keep their servers.")
        r_banquet_room = ("Banquet Room", "This looks like a room for large events.")
        r_nest = ("Nest", "This looks web filled nest for some kind of creature.")
        r_armory = ("Armory", "This stone room is full of arms. To Arms!")
        r_cave = ("Cave", "This stone room looks to be a natural cave opening.")
        r_mine = ("Mine", "This area looks to be mine set up to find coal, iron, diamonds...and redstone...Punch a tree and get your torches ready!")
        r_office = ("Office", "This room looks like a basement office at the state department.")           
        r_walk_in_closet = ("Walk In Closet", "This room is a huge walk-in-closet. There's clothing everywhere! What????")
        r_ball_room = ("Ball Room", "This room is full to waist height of small plastic multi-colored balls...be careful!!!")
        # List of possible room types of Rooms
        room_type_list = [r_long_hallway, r_medium_hallway, r_short_hallway, r_stone_room, r_wooden_room, r_swimming_pool, r_library, r_storage_room, r_kitchen, r_dungeon_cell, r_server_room, r_banquet_room, r_nest, r_armory, r_mine, r_office, r_walk_in_closet, r_ball_room]
        # set values
        x = 0
        y = 0
        current_direction = 'n'
        # while look for making rooms
        # Random walk through grid of rooms, 
        # any direction except backwards
        while room_count < num_rooms:
            # randomly pick room type
            this_room_type = random.choice(room_type_list)
            # # inspection
            # print(this_room_type)
            # previous_room_direction = reverse_dirs[current_direction]
            possible_directions = ['n', 's', 'e' ,'w']
            # starts with a random choice:
            current_direction = random.choice(possible_directions)
            # save old values
            x_old = x
            y_old = y
            # save the location of the past room
            past_room = (x,y)
            # "don't go out of bounds"
            # check to make sure not out of bounds
            # while out of bounds, keep trying again for a not-out option
            while (x >= size_x - 1) or (y >= size_y - 1) or (x < 0) or (y < 0) or ((x,y) == past_room) or ((x,y) == (0,0)):
                # save old values
                x = x_old
                y = y_old
                # starts with a random choice:
                current_direction = random.choice(possible_directions)
                if current_direction == 'n':
                    y += 1
                if current_direction == 's':
                    y -= 1
                if current_direction == 'e':
                    x += 1
                if current_direction == 'w':
                    x -= 1
            # # inspection
            # print("room picked", x,y)
            # look for a previously made room (where you are looking 
            # to make a new room)
            # if not (already one there), make a new room
            if self.grid[y][x] is None:
                # Create a room in the given direction
                maker_room = Room_maker(room_count, this_room_type[0], this_room_type[1], x, y)
                # room = Room(room_count, this_room_type[0], this_room_type[1])
                room = Room(room_count, this_room_type[0], this_room_type[1], x, y)   
                # Save the room in the World grid
                self.grid[y][x] = maker_room
                # update room counter, only if a new room is made
                room_count += 1
                # add room to room list:
                mask_dict = {'x':x, 'y':y}
                self.rooms_list.extend([mask_dict])
            # sets previous reverse room direction
            reverse_dirs = {"n": "s", "s": "n", "e": "w", "w": "e"}
            previous_room_direction = reverse_dirs[current_direction]
            # either way: Connect the new room to the previous room
            if previous_room is not None:
                # connect to the previous room...in DB
                previous_room.maker_connect_rooms(maker_room, previous_room_direction)
                maker_room.maker_connect_rooms(previous_room, current_direction)
                # # connect to the previous room...in room maker
                previous_room.connect_rooms(room, previous_room_direction)
                room.connect_rooms(previous_room, current_direction)
                # doors list: add room to list:
                mask = [{'x': int(maker_room.x), 'y': int(maker_room.y)}, {'x': int(previous_room.x), 'y': int(previous_room.y)}]
                self.doors_list.extend([mask])
            # Update iteration variables
            previous_room = maker_room
            # Update iteration variables
            previous_room = maker_room
            # # save room to django datatable
            room.save()
# # generate world:
# # set input values
w = World()
num_rooms = 100
width = 20
height = 20
w.generate_rooms(width, height, num_rooms)
players = Player.objects.all()
for p in players:
    p.currentRoom = r_outside.id
    p.save()

In [0]:
# create_world.py / this version only works in heroku
# this is the file that must be run, which dungeon making must be merged into
from django.contrib.auth.models import User
from adventure.models import Player, Room
from adventure.classes import Room_maker, World
# steps:
# step 1: make rooms
# step 2: make players
#
# erases pre-existing rooms
Room.objects.all().delete()
# creates set rooms
# randomly generated/(or selected-from-smaller-set) description
# merging code
# known bug: may loop if dungeon-grid size is too small
# 2 lists: generated as the rooms are made
# list of rooms  [{x: 4, y: 5},{x: 4, y: 5}, ...]
# list of connections  [ [{x: 4, y: 5}, {x: 4, y: 5}], 
#                        [{x: 4, y: 5}, {x: 4, y: 5}] ]
class Room_maker:  # Class: Room input parameters (id, name, description, x, y)
    # constructor
    def __init__(self, id, name, description, x, y):
        # attributes
        self.id = id
        self.name = name
        self.description = description
        self.n_to = None
        self.s_to = None
        self.e_to = None
        self.w_to = None
        self.x = x
        self.y = y
    # ?
    # class methods
    def __repr__(self):
        # if self.e_to is not None:
        #     return f"({self.x}, {self.y}) -> ({self.e_to.x}, {self.e_to.y})"
        return f"({self.x}, {self.y})"
    # connects to the previously make room
    def maker_connect_rooms(self, connecting_room, direction):
        # Connect two rooms in the given n/s/e/w direction
        reverse_dirs = {"n": "s", "s": "n", "e": "w", "w": "e"}
        reverse_dir = reverse_dirs[direction]
        setattr(self, f"{direction}_to", connecting_room)
        setattr(connecting_room, f"{reverse_dir}_to", self)
    # this is just for the player
    def get_room_in_direction(self, direction):
        return getattr(self, f"{direction}_to")
# see while loop for making random rooms
class World:  # Class
    # constructor
    def __init__(self):
        # attributes of world-grid
        self.grid = None
        # # 2 lists for front end mapping
        self.rooms_list = None
        self.doors_list = None
        self.width = 0
        self.height = 0
        self.first_room = None
    # class methods    
    def generate_rooms(self, size_x, size_y, num_rooms):
        # Step 1: set up variables and things for while loop
        # Step 2: run while loop to create rooms
        # this makes any empty world grid - why this sequence?
        #
        # make part of grid? sets y dimension?
        self.grid = [None] * size_y
        # sets max size of world-grid
        self.width = size_x        
        self.height = size_y
        # ? since grid was made, what does this do?
        # sets x dimension?
        for i in range( len(self.grid) ):
            self.grid[i] = [None] * size_x
        #
        # # 2 lists for front end mapping
        self.rooms_list = []
        self.doors_list = []
        # Start from lower-left corner (0,0)
        x = -1 # (this will become 0 on the first step)
        y = 0
        # keeping values for comparison
        previous_id_tuple = None
        previous_room = None
        previous_direction = None
        # up to max number of rooms
        room_count = 0
        # move these out of the while loop
        import random
        # standard room types
        r_long_hallway = ("Long Hallway", "This hallway is very long. Be careful.")
        r_short_hallway = ("Short Hallway", "This hallway is not very long. Be very careful.")
        r_medium_hallway = ("Medium Hallway", "This hallway is... Be very careful.")
        r_stone_room = ("Stone Room", "This stone room is dark. Be careful.")
        r_wooden_room = ("Wooden Room", "The wooden room is very...woodish.")
        r_swimming_pool = ("Swimming Pool", "This looks like a great place to swim...is that a giant squid?")
        r_library = ("library", "This library is very bright and quiet.")
        r_storage_room = ("storage_room", "This storage room is full of things.")
        r_kitchen = ("Kitchen", "This room is full of cooking supplies.")
        r_dungeon_cell = ("Dungeon Cell", "This room looks like a prison cell.")
        r_server_room = ("Dungeon Server Wearhouse", "This where the monsters keep their servers.")
        r_banquet_room = ("Banquet Room", "This looks like a room for large events.")
        r_nest = ("Nest", "This looks web filled nest for some kind of creature.")
        r_armory = ("Armory", "This stone room is full of arms. To Arms!")
        r_cave = ("Cave", "This stone room looks to be a natural cave opening.")
        r_mine = ("Mine", "This area looks to be mine set up to find coal, iron, diamonds...and redstone...Punch a tree and get your torches ready!")
        r_office = ("Office", "This room looks like a basement office at the state department.")           
        r_walk_in_closet = ("Walk In Closet", "This room is a huge walk-in-closet. There's clothing everywhere! What????")
        r_ball_room = ("Ball Room", "This room is full to waist height of small plastic multi-colored balls...be careful!!!")
        # List of possible room types of Rooms
        room_type_list = [r_long_hallway, r_medium_hallway, r_short_hallway, r_stone_room, r_wooden_room, r_swimming_pool, r_library, r_storage_room, r_kitchen, r_dungeon_cell, r_server_room, r_banquet_room, r_nest, r_armory, r_mine, r_office, r_walk_in_closet, r_ball_room]
        # set values
        x = 0
        y = 0
        current_direction = 'n'
        # while look for making rooms
        # Random walk through grid of rooms, 
        # any direction except backwards
        while room_count < num_rooms:
            # randomly pick room type
            this_room_type = random.choice(room_type_list)
            # # inspection
            # print(this_room_type)
            # previous_room_direction = reverse_dirs[current_direction]
            possible_directions = ['n', 's', 'e' ,'w']
            # starts with a random choice:
            current_direction = random.choice(possible_directions)
            # save old values
            x_old = x
            y_old = y
            # save the location of the past room
            past_room = (x,y)
            # "don't go out of bounds"
            # check to make sure not out of bounds
            # while out of bounds, keep trying again for a not-out option
            while (x >= size_x - 1) or (y >= size_y - 1) or (x < 0) or (y < 0) or ((x,y) == past_room) or ((x,y) == (0,0)):
                # save old values
                x = x_old
                y = y_old
                # starts with a random choice:
                current_direction = random.choice(possible_directions)
                if current_direction == 'n':
                    y += 1
                if current_direction == 's':
                    y -= 1
                if current_direction == 'e':
                    x += 1
                if current_direction == 'w':
                    x -= 1
            # # inspection
            # print("room picked", x,y)
            # look for a previously made room (where you are looking 
            # to make a new room)
            # if not (already one there), make a new room
            if self.grid[y][x] is None:
                # Create a room in the given direction
                maker_room = Room_maker(room_count, this_room_type[0], this_room_type[1], x, y)
                # room = Room(room_count, this_room_type[0], this_room_type[1])
                room = Room(room_count, this_room_type[0], this_room_type[1], x, y)   
                # Save the room in the World grid
                self.grid[y][x] = maker_room
                # update room counter, only if a new room is made
                room_count += 1
                # add room to room list:
                mask_dict = {'x':x, 'y':y}
                self.rooms_list.extend([mask_dict])
            # sets previous reverse room direction
            reverse_dirs = {"n": "s", "s": "n", "e": "w", "w": "e"}
            previous_room_direction = reverse_dirs[current_direction]
            # either way: Connect the new room to the previous room
            if previous_room is not None:
                # connect to the previous room...in DB
                previous_room.maker_connect_rooms(maker_room, previous_room_direction)
                maker_room.maker_connect_rooms(previous_room, current_direction)
                # # connect to the previous room...in room maker
                previous_room.connect_rooms(room, previous_room_direction)
                room.connect_rooms(previous_room, current_direction)
                # doors list: add room to list:
                mask = [{'x': int(maker_room.x), 'y': int(maker_room.y)}, {'x': int(previous_room.x), 'y': int(previous_room.y)}]
                self.doors_list.extend([mask])
            # Update iteration variables
            previous_room = maker_room
            # Update iteration variables
            previous_room = maker_room
            #
            if room_count == 1:
                self.first_room = room
            # # save room to django datatable
            room.save()
# # generate world:
# # set input values
w = World()
num_rooms = 100
width = 20
height = 20
w.generate_rooms(width, height, num_rooms)
players = Player.objects.all()
for p in players:
    p.currentRoom = w.first_room.id
    p.save()

In [0]:
from django.contrib.auth.models import User
from adventure.models import Player, Room

# creates set rooms
# randomly generated/(or selected-from-smaller-set) description
# merging code
# known bug: may loop if dungeon-grid size is too small
# 2 lists: generated as the rooms are made
# list of rooms  [{x: 4, y: 5},{x: 4, y: 5}, ...]
# list of connections  [ [{x: 4, y: 5}, {x: 4, y: 5}], 
#                        [{x: 4, y: 5}, {x: 4, y: 5}] ]

class Room_maker:  # Class: Room input parameters (id, name, description, x, y)
    # constructor
    def __init__(self, id, name, description, x, y):
        # attributes
        self.id = id
        self.name = name
        self.description = description
        self.n_to = None
        self.s_to = None
        self.e_to = None
        self.w_to = None
        self.x = x
        self.y = y

    # ?
    # class methods
    def __repr__(self):
        # if self.e_to is not None:
        #     return f"({self.x}, {self.y}) -> ({self.e_to.x}, {self.e_to.y})"
        return f"({self.x}, {self.y})"

    # connects to the previously make room
    def maker_connect_rooms(self, connecting_room, direction):
        # Connect two rooms in the given n/s/e/w direction
        reverse_dirs = {"n": "s", "s": "n", "e": "w", "w": "e"}
        reverse_dir = reverse_dirs[direction]
        setattr(self, f"{direction}_to", connecting_room)
        setattr(connecting_room, f"{reverse_dir}_to", self)

    # this is just for the player
    def get_room_in_direction(self, direction):
        return getattr(self, f"{direction}_to")

# see while loop for making random rooms
class World:  # Class

    # constructor
    def __init__(self):
        # attributes of world-grid
        self.grid = None
        # # 2 lists for front end mapping
        self.rooms_list = None
        self.doors_list = None
        self.width = 0
        self.height = 0
        self.first_room = None

    # class methods    
    def generate_rooms(self, size_x, size_y, num_rooms):
        # Step 1: set up variables and things for while loop
        # Step 2: run while loop to create rooms
        # this makes any empty world grid - why this sequence?
        #
        # make part of grid? sets y dimension?
        self.grid = [None] * size_y
        # sets max size of world-grid
        self.width = size_x        
        self.height = size_y
        # ? since grid was made, what does this do?
        # sets x dimension?
        for i in range( len(self.grid) ):
            self.grid[i] = [None] * size_x
        #
        # # 2 lists for front end mapping
        self.rooms_list = []
        self.doors_list = []

        # Start from lower-left corner (0,0)
        x = -1 # (this will become 0 on the first step)
        y = 0

        # keeping values for comparison
        previous_id_tuple = None
        previous_room = None
        previous_direction = None

        # up to max number of rooms
        room_count = 0

        # move these out of the while loop
        import random

        # for random room generation
        # standard room types
        r_long_hallway = ("Long Hallway", "This hallway is very long. Be careful.")
        r_short_hallway = ("Short Hallway", "This hallway is not very long. Be very careful.")
        r_medium_hallway = ("Medium Hallway", "This hallway is... Be very careful.")
        r_stone_room = ("Stone Room", "This stone room is dark. Be careful.")
        r_wooden_room = ("Wooden Room", "The wooden room is very...woodish.")
        r_swimming_pool = ("Swimming Pool", "This looks like a great place to swim...is that a giant squid?")
        r_library = ("library", "This library is very bright and quiet.")
        r_storage_room = ("storage_room", "This storage room is full of things.")
        r_kitchen = ("Kitchen", "This room is full of cooking supplies.")
        r_dungeon_cell = ("Dungeon Cell", "This room looks like a prison cell.")
        r_server_room = ("Dungeon Server Wearhouse", "This where the monsters keep their servers.")
        r_banquet_room = ("Banquet Room", "This looks like a room for large events.")
        r_nest = ("Nest", "This looks web filled nest for some kind of creature.")
        r_armory = ("Armory", "This stone room is full of arms. To Arms!")
        r_cave = ("Cave", "This stone room looks to be a natural cave opening.")
        r_mine = ("Mine", "This area looks to be mine set up to find coal, iron, diamonds...and redstone...Punch a tree and get your torches ready!")
        r_office = ("Office", "This room looks like a basement office at the state department.")           
        r_walk_in_closet = ("Walk In Closet", "This room is a huge walk-in-closet. There's clothing everywhere! What????")
        r_ball_room = ("Ball Room", "This room is full to waist height of small plastic multi-colored balls...be careful!!!")
        # List of possible room types of Rooms
        room_type_list = [r_long_hallway, r_medium_hallway, r_short_hallway, r_stone_room, r_wooden_room, r_swimming_pool, r_library, r_storage_room, r_kitchen, r_dungeon_cell, r_server_room, r_banquet_room, r_nest, r_armory, r_mine, r_office, r_walk_in_closet, r_ball_room]

        # set values
        x = 0
        y = 0
        current_direction = 'n'

        # while look for making rooms
        # Random walk through grid of rooms, 
        # any direction except backwards
        while room_count < num_rooms:
            # randomly pick room type
            this_room_type = random.choice(room_type_list)

            # # inspection
            # print(this_room_type)
            # previous_room_direction = reverse_dirs[current_direction]
            possible_directions = ['n', 's', 'e' ,'w']

            # starts with a random choice:
            current_direction = random.choice(possible_directions)

            # save old values
            x_old = x
            y_old = y

            # save the location of the past room
            past_room = (x,y)

            # "don't go out of bounds"
            # check to make sure not out of bounds
            # while out of bounds, keep trying again for a not-out option
            while (x >= size_x - 1) or (y >= size_y - 1) or (x < 0) or (y < 0) or ((x,y) == past_room) or ((x,y) == (0,0)):
                # save old values
                x = x_old
                y = y_old
                # starts with a random choice:
                current_direction = random.choice(possible_directions)
                if current_direction == 'n':
                    y += 1
                if current_direction == 's':
                    y -= 1
                if current_direction == 'e':
                    x += 1
                if current_direction == 'w':
                    x -= 1

            # look for a previously made room (where you are looking 
            # to make a new room)
            # if not (already one there), make a new room
            if self.grid[y][x] is None:
                # Create a room in the given direction
                maker_room = Room_maker(room_count, this_room_type[0], this_room_type[1], x, y)
                # room = Room(room_count, this_room_type[0], this_room_type[1])
                room = Room(room_count, this_room_type[0], this_room_type[1], x, y)   
                # Save the room in the World grid
                self.grid[y][x] = maker_room
                # update room counter, only if a new room is made
                room_count += 1
                # add room to room list:
                mask_dict = {'x':x, 'y':y}
                self.rooms_list.extend([mask_dict])

            # sets previous reverse room direction
            reverse_dirs = {"n": "s", "s": "n", "e": "w", "w": "e"}
            previous_room_direction = reverse_dirs[current_direction]

            # either way: Connect the new room to the previous room
            if previous_room is not None:
                # connect to the previous room...in DB
                previous_room.maker_connect_rooms(maker_room, previous_room_direction)
                maker_room.maker_connect_rooms(previous_room, current_direction)
                # # connect to the previous room...in room maker
                previous_room.connect_rooms(room, previous_room_direction)
                room.connect_rooms(previous_room, current_direction)
                # doors list: add room to list:
                mask = [{'x': int(maker_room.x), 'y': int(maker_room.y)}, {'x': int(previous_room.x), 'y': int(previous_room.y)}]
                self.doors_list.extend([mask])

            # Update iteration variables
            previous_room = maker_room

            # Update iteration variables
            previous_room = maker_room
            
            # Saves the room-object for just the first room
            # to be used later as the starting location of
            # the player.
            if room_count == 1:
                self.first_room = room

            # # saves the room to django datatable
            # # or just makes everything break
            room.save()



In [0]:
# from django.contrib.auth.models import User
# from adventure.models import Player, Room

# creates set rooms
# randomly generated/(or selected-from-smaller-set) description
# merging code
# known bug: may loop if dungeon-grid size is too small
# 2 lists: generated as the rooms are made
# list of rooms  [{x: 4, y: 5},{x: 4, y: 5}, ...]
# list of connections  [ [{x: 4, y: 5}, {x: 4, y: 5}], 
#                        [{x: 4, y: 5}, {x: 4, y: 5}] ]

class Room_maker:  # Class: Room input parameters (id, name, description, x, y)
    # constructor
    def __init__(self, id, name, description, x, y):
        # attributes
        self.id = id
        self.name = name
        self.description = description
        self.n_to = None
        self.s_to = None
        self.e_to = None
        self.w_to = None
        self.x = x
        self.y = y

    # ?
    # class methods
    def __repr__(self):
        # if self.e_to is not None:
        #     return f"({self.x}, {self.y}) -> ({self.e_to.x}, {self.e_to.y})"
        return f"({self.x}, {self.y})"

    # connects to the previously make room
    def maker_connect_rooms(self, connecting_room, direction):
        # Connect two rooms in the given n/s/e/w direction
        reverse_dirs = {"n": "s", "s": "n", "e": "w", "w": "e"}
        reverse_dir = reverse_dirs[direction]
        setattr(self, f"{direction}_to", connecting_room)
        setattr(connecting_room, f"{reverse_dir}_to", self)

    # this is just for the player
    def get_room_in_direction(self, direction):
        return getattr(self, f"{direction}_to")

# see while loop for making random rooms
class World:  # Class

    # constructor
    def __init__(self):
        # attributes of world-grid
        self.grid = None
        # # 2 lists for front end mapping
        self.rooms_list = None
        self.doors_list = None
        self.width = 0
        self.height = 0
        self.first_room = None
        self.room_obj_list = []
        self.room_connections_list = []

    # class methods    
    def generate_rooms(self, size_x, size_y, num_rooms):
        # Step 1: set up variables and things for while loop
        # Step 2: run while loop to create rooms
        # this makes any empty world grid - why this sequence?
        #
        # make part of grid? sets y dimension?
        self.grid = [None] * size_y
        # sets max size of world-grid
        self.width = size_x        
        self.height = size_y
        # ? since grid was made, what does this do?
        # sets x dimension?
        for i in range( len(self.grid) ):
            self.grid[i] = [None] * size_x
        #
        # # 2 lists for front end mapping
        self.rooms_list = []
        self.doors_list = []

        # Start from lower-left corner (0,0)
        x = -1 # (this will become 0 on the first step)
        y = 0

        # keeping values for comparison
        previous_id_tuple = None
        previous_room = None
        previous_direction = None

        # up to max number of rooms
        room_count = 0

        # move these out of the while loop
        import random

        # for random room generation
        # standard room types
        r_long_hallway = ("Long Hallway", "This hallway is very long. Be careful.")
        r_short_hallway = ("Short Hallway", "This hallway is not very long. Be very careful.")
        r_medium_hallway = ("Medium Hallway", "This hallway is... Be very careful.")
        r_stone_room = ("Stone Room", "This stone room is dark. Be careful.")
        r_wooden_room = ("Wooden Room", "The wooden room is very...woodish.")
        r_swimming_pool = ("Swimming Pool", "This looks like a great place to swim...is that a giant squid?")
        r_library = ("library", "This library is very bright and quiet.")
        r_storage_room = ("storage_room", "This storage room is full of things.")
        r_kitchen = ("Kitchen", "This room is full of cooking supplies.")
        r_dungeon_cell = ("Dungeon Cell", "This room looks like a prison cell.")
        r_server_room = ("Dungeon Server Wearhouse", "This where the monsters keep their servers.")
        r_banquet_room = ("Banquet Room", "This looks like a room for large events.")
        r_nest = ("Nest", "This looks web filled nest for some kind of creature.")
        r_armory = ("Armory", "This stone room is full of arms. To Arms!")
        r_cave = ("Cave", "This stone room looks to be a natural cave opening.")
        r_mine = ("Mine", "This area looks to be mine set up to find coal, iron, diamonds...and redstone...Punch a tree and get your torches ready!")
        r_office = ("Office", "This room looks like a basement office at the state department.")           
        r_walk_in_closet = ("Walk In Closet", "This room is a huge walk-in-closet. There's clothing everywhere! What????")
        r_ball_room = ("Ball Room", "This room is full to waist height of small plastic multi-colored balls...be careful!!!")
        # List of possible room types of Rooms
        room_type_list = [r_long_hallway, r_medium_hallway, r_short_hallway, r_stone_room, r_wooden_room, r_swimming_pool, r_library, r_storage_room, r_kitchen, r_dungeon_cell, r_server_room, r_banquet_room, r_nest, r_armory, r_mine, r_office, r_walk_in_closet, r_ball_room]

        # set values
        x = 0
        y = 0
        current_direction = 'n'

        # while look for making rooms
        # Random walk through grid of rooms, 
        # any direction except backwards
        while room_count < num_rooms:
            # randomly pick room type
            this_room_type = random.choice(room_type_list)

            # # inspection
            # print(this_room_type)
            # previous_room_direction = reverse_dirs[current_direction]
            possible_directions = ['n', 's', 'e' ,'w']

            # starts with a random choice:
            current_direction = random.choice(possible_directions)

            # save old values
            x_old = x
            y_old = y

            # save the location of the past room
            past_room = (x,y)

            # "don't go out of bounds"
            # check to make sure not out of bounds
            # while out of bounds, keep trying again for a not-out option
            while (x >= size_x - 1) or (y >= size_y - 1) or (x < 0) or (y < 0) or ((x,y) == past_room) or ((x,y) == (0,0)):
                # save old values
                x = x_old
                y = y_old
                # starts with a random choice:
                current_direction = random.choice(possible_directions)
                if current_direction == 'n':
                    y += 1
                if current_direction == 's':
                    y -= 1
                if current_direction == 'e':
                    x += 1
                if current_direction == 'w':
                    x -= 1

            # look for a previously made room (where you are looking 
            # to make a new room)
            # if not (already one there), make a new room
            if self.grid[y][x] is None:
                # Create a room in the given direction
                maker_room = Room_maker(room_count, this_room_type[0], this_room_type[1], x, y)
                # room = Room(room_count, this_room_type[0], this_room_type[1])
                room = Room(room_count, this_room_type[0], this_room_type[1], x, y)   
                # Save the room in the World grid
                self.grid[y][x] = maker_room
                # update room counter, only if a new room is made
                room_count += 1
                # add room to room list:
                mask_dict = {'x':x, 'y':y}
                self.rooms_list.extend([mask_dict])

            # sets previous reverse room direction
            reverse_dirs = {"n": "s", "s": "n", "e": "w", "w": "e"}
            previous_room_direction = reverse_dirs[current_direction]

            # either way: Connect the new room to the previous room
            if previous_room is not None:
                # connect to the previous room...in DB
                previous_room.maker_connect_rooms(maker_room, previous_room_direction)
                maker_room.maker_connect_rooms(previous_room, current_direction)

                # # # connect to the previous room...in room maker
                # previous_room.connect_rooms(room, previous_room_direction)
                # room.connect_rooms(previous_room, current_direction)

                # doors list: add room to list:
                mask = [{'x': int(maker_room.x), 'y': int(maker_room.y)}, {'x': int(previous_room.x), 'y': int(previous_room.y)}]
                self.doors_list.extend([mask])

                # this saves a:  
                # will this work? pull up each tuple and slice that?
                mask3 = (maker_room , previous_room)
                self.doors_list.extend(mask)

            # Update iteration variables
            previous_room = maker_room
            
            # Saves the room-object for just the first room
            # to be used later as the starting location of
            # the player.
            if room_count == 1:
                self.first_room = room

            # # saves the room to django datatable
            # # or just makes everything break
            self.room_object_list.extend(room) 
            #room.save()

    return self.room_object_list



function-onely version

In [49]:
# # from django.contrib.auth.models import User
# # from adventure.models import Player, Room

# # creates set rooms
# # randomly generated/(or selected-from-smaller-set) description
# # merging code
# # known bug: may loop if dungeon-grid size is too small
# # 2 lists: generated as the rooms are made
# # list of rooms  [{x: 4, y: 5},{x: 4, y: 5}, ...]
# # list of connections  [ [{x: 4, y: 5}, {x: 4, y: 5}], 
# #                        [{x: 4, y: 5}, {x: 4, y: 5}] ]

# # class Room_maker:  # Class: Room input parameters (id, name, description, x, y)
# #     # constructor
# #     def __init__(self, id, name, description, x, y):
# #         # attributes
# #         self.id = id
# #         self.name = name
# #         self.description = description
# #         self.n_to = None
# #         self.s_to = None
# #         self.e_to = None
# #         self.w_to = None
# #         self.x = x
# #         self.y = y

# #     # connects to the previously make room
# #     def maker_connect_rooms(self, connecting_room, direction):
# #         # Connect two rooms in the given n/s/e/w direction
# #         reverse_dirs = {"n": "s", "s": "n", "e": "w", "w": "e"}
# #         reverse_dir = reverse_dirs[direction]
# #         setattr(self, f"{direction}_to", connecting_room)
# #         setattr(connecting_room, f"{reverse_dir}_to", self)


# class Room():
#     title = "DEFAULT TITLE"
#     description = "DEFAULT DESCRIPTION"
#     n_to = 0
#     s_to = 0
#     e_to = 0
#     w_to = 0
#     x = 0
#     y = 0
    
# # class Room(models.Model):
# #     title = models.CharField(max_length=50, default="DEFAULT TITLE")
# #     description = models.CharField(max_length=500, default="DEFAULT DESCRIPTION")
# #     n_to = models.IntegerField(default=0)
# #     s_to = models.IntegerField(default=0)
# #     e_to = models.IntegerField(default=0)
# #     w_to = models.IntegerField(default=0)
# #     x = models.IntegerField(default=0)
# #     y = models.IntegerField(default=0)

#     def connectRooms(self, destinationRoom, direction):
#         destinationRoomID = destinationRoom.id
#         try:
#             destinationRoom = Room.objects.get(id=destinationRoomID)
#         except Room.DoesNotExist:
#             print("That room does not exist")
#         else:
#             if direction == "n":
#                 self.n_to = destinationRoomID
#             elif direction == "s":
#                 self.s_to = destinationRoomID
#             elif direction == "e":
#                 self.e_to = destinationRoomID
#             elif direction == "w":
#                 self.w_to = destinationRoomID
#             else:
#                 print("Invalid direction")
#                 return
#             self.save()
#     # def playerNames(self, currentPlayerID):
#     #     return [p.user.username for p in Player.objects.filter(currentRoom=self.id) if p.id != int(currentPlayerID)]
#     # def playerUUIDs(self, currentPlayerID):
#     #     return [p.uuid for p in Player.objects.filter(currentRoom=self.id) if p.id != int(currentPlayerID)]



#
# World
#
# attributes of world-grid
grid = None
# # 2 lists for front end mapping
rooms_list = None
doors_list = None
width = 0
height = 0
first_room = None
room_obj_list = []
room_connections_list = []
room_obj_list = []

# set input values
num_rooms = 100
width = 20
size_x = width
height = 20
size_y = height

#def generate_rooms(size_x, size_y, num_rooms):
# Step 1: set up variables and things for while loop
# Step 2: run while loop to create rooms
# this makes any empty world grid - why this sequence?
#
# make part of grid? sets y dimension?
grid = [None] * size_y
# sets max size of world-grid
width = size_x        
height = size_y
# ? since grid was made, what does this do?
# sets x dimension?
for i in range( len(grid) ):
    grid[i] = [None] * size_x
#
# # 2 lists for front end mapping
rooms_list = []
doors_list = []

# Start from lower-left corner (0,0)
x = -1 # (this will become 0 on the first step)
y = 0

# keeping values for comparison
previous_id_tuple = None
previous_room = None
previous_direction = None

# up to max number of rooms
room_count = 0

# move these out of the while loop
import random

# for random room generation
# standard room types
r_long_hallway = ("Long Hallway", "This hallway is very long. Be careful.")
r_short_hallway = ("Short Hallway", "This hallway is not very long. Be very careful.")
r_medium_hallway = ("Medium Hallway", "This hallway is... Be very careful.")
r_stone_room = ("Stone Room", "This stone room is dark. Be careful.")
r_wooden_room = ("Wooden Room", "The wooden room is very...woodish.")
r_swimming_pool = ("Swimming Pool", "This looks like a great place to swim...is that a giant squid?")
r_library = ("library", "This library is very bright and quiet.")
r_storage_room = ("storage_room", "This storage room is full of things.")
r_kitchen = ("Kitchen", "This room is full of cooking supplies.")
r_dungeon_cell = ("Dungeon Cell", "This room looks like a prison cell.")
r_server_room = ("Dungeon Server Wearhouse", "This where the monsters keep their servers.")
r_banquet_room = ("Banquet Room", "This looks like a room for large events.")
r_nest = ("Nest", "This looks web filled nest for some kind of creature.")
r_armory = ("Armory", "This stone room is full of arms. To Arms!")
r_cave = ("Cave", "This stone room looks to be a natural cave opening.")
r_mine = ("Mine", "This area looks to be mine set up to find coal, iron, diamonds...and redstone...Punch a tree and get your torches ready!")
r_office = ("Office", "This room looks like a basement office at the state department.")           
r_walk_in_closet = ("Walk In Closet", "This room is a huge walk-in-closet. There's clothing everywhere! What????")
r_ball_room = ("Ball Room", "This room is full to waist height of small plastic multi-colored balls...be careful!!!")
# List of possible room types of Rooms
room_type_list = [r_long_hallway, r_medium_hallway, r_short_hallway, r_stone_room, r_wooden_room, r_swimming_pool, r_library, r_storage_room, r_kitchen, r_dungeon_cell, r_server_room, r_banquet_room, r_nest, r_armory, r_mine, r_office, r_walk_in_closet, r_ball_room]

# set values
x = 0
y = 0
current_direction = 'n'

# while look for making rooms
# Random walk through grid of rooms, 
# any direction except backwards
while room_count < num_rooms:
    # randomly pick room type
    this_room_type = random.choice(room_type_list)

    # # inspection
    # print(this_room_type)
    # previous_room_direction = reverse_dirs[current_direction]
    possible_directions = ['n', 's', 'e' ,'w']

    # starts with a random choice:
    current_direction = random.choice(possible_directions)

    # save old values
    x_old = x
    y_old = y

    # save the location of the past room
    past_room = (x,y)

    # "don't go out of bounds"
    # check to make sure not out of bounds
    # while out of bounds, keep trying again for a not-out option
    while (x >= size_x - 1) or (y >= size_y - 1) or (x < 0) or (y < 0) or ((x,y) == past_room) or ((x,y) == (0,0)):
        # save old values
        x = x_old
        y = y_old
        # starts with a random choice:
        current_direction = random.choice(possible_directions)
        if current_direction == 'n':
            y += 1
        if current_direction == 's':
            y -= 1
        if current_direction == 'e':
            x += 1
        if current_direction == 'w':
            x -= 1

    # look for a previously made room (where you are looking 
    # to make a new room)
    # if not (already one there), make a new room
    if grid[y][x] is None:
        # # Create a room in the given direction
        # room = Room(room_count, this_room_type[0], this_room_type[1])
        room = Room(this_room_type[0], this_room_type[1], x, y)   
        # Save the room in the World grid
        grid[y][x] = room
        # update room counter, only if a new room is made
        room_count += 1
        # add room to room list:
        mask_dict = {'x':x, 'y':y}
        rooms_list.extend([mask_dict])

    # sets previous reverse room direction
    reverse_dirs = {"n": "s", "s": "n", "e": "w", "w": "e"}
    previous_room_direction = reverse_dirs[current_direction]

    # either way: Connect the new room to the previous room
    if previous_room is not None:
        # connect to the previous room...in DB
        previous_room.maker_connect_rooms(room, previous_room_direction)
        room.maker_connect_rooms(previous_room, current_direction)

        # # # connect to the previous room...in room maker
        # previous_room.connect_rooms(room, previous_room_direction)
        # room.connect_rooms(previous_room, current_direction)

        # doors list: add room to list:
        mask = [{'x': int(room.x), 'y': int(room.y)}, {'x': int(previous_room.x), 'y': int(previous_room.y)}]
        doors_list.extend([mask])

        # this saves a:  
        # will this work? pull up each tuple and slice that?
        mask3 = (room , previous_room)
        doors_list.extend(mask)

    # Update iteration variables
    previous_room = room
    
    # Saves the room-object for just the first room
    # to be used later as the starting location of
    # the player.
    if room_count == 1:
        first_room = room

    # # saves the room to django datatable
    # # or just makes everything break
    #room_obj_list.extend(room) 
    room.save()

# num_rooms = 100
# width = 20
# height = 20
# generate_rooms(width, height, num_rooms)

#print(room_obj_list)
print(rooms_list)
print(doors_list)
#print(grid)


TypeError: ignored

In [0]:
from django.contrib.auth.models import User
from adventure.models import Player, Room
# World
#
# attributes of world-grid
grid = None
# # 2 lists for front end mapping
rooms_list = None
doors_list = None
width = 0
height = 0
first_room = None
room_obj_list = []
room_connections_list = []
room_obj_list = []
# set input values
num_rooms = 100
width = 20
size_x = width
height = 20
size_y = height
#def generate_rooms(size_x, size_y, num_rooms):
# Step 1: set up variables and things for while loop
# Step 2: run while loop to create rooms
# this makes any empty world grid - why this sequence?
#
# make part of grid? sets y dimension?
grid = [None] * size_y
# sets max size of world-grid
width = size_x        
height = size_y
# ? since grid was made, what does this do?
# sets x dimension?
for i in range( len(grid) ):
    grid[i] = [None] * size_x
# # 2 lists for front end mapping
rooms_list = []
doors_list = []
# Start from lower-left corner (0,0)
x = -1 # (this will become 0 on the first step)
y = 0
# keeping values for comparison
previous_id_tuple = None
previous_room = None
previous_direction = None
# up to max number of rooms
room_count = 0
# move these out of the while loop
import random
# for random room generation
# standard room types
r_long_hallway = ("Long Hallway", "This hallway is very long. Be careful.")
r_short_hallway = ("Short Hallway", "This hallway is not very long. Be very careful.")
r_medium_hallway = ("Medium Hallway", "This hallway is... Be very careful.")
r_stone_room = ("Stone Room", "This stone room is dark. Be careful.")
r_wooden_room = ("Wooden Room", "The wooden room is very...woodish.")
r_swimming_pool = ("Swimming Pool", "This looks like a great place to swim...is that a giant squid?")
r_library = ("library", "This library is very bright and quiet.")
r_storage_room = ("storage_room", "This storage room is full of things.")
r_kitchen = ("Kitchen", "This room is full of cooking supplies.")
r_dungeon_cell = ("Dungeon Cell", "This room looks like a prison cell.")
r_server_room = ("Dungeon Server Wearhouse", "This where the monsters keep their servers.")
r_banquet_room = ("Banquet Room", "This looks like a room for large events.")
r_nest = ("Nest", "This looks web filled nest for some kind of creature.")
r_armory = ("Armory", "This stone room is full of arms. To Arms!")
r_cave = ("Cave", "This stone room looks to be a natural cave opening.")
r_mine = ("Mine", "This area looks to be mine set up to find coal, iron, diamonds...and redstone...Punch a tree and get your torches ready!")
r_office = ("Office", "This room looks like a basement office at the state department.")           
r_walk_in_closet = ("Walk In Closet", "This room is a huge walk-in-closet. There's clothing everywhere! What????")
r_ball_room = ("Ball Room", "This room is full to waist height of small plastic multi-colored balls...be careful!!!")
# List of possible room types of Rooms
room_type_list = [r_long_hallway, r_medium_hallway, r_short_hallway, r_stone_room, r_wooden_room, r_swimming_pool, r_library, r_storage_room, r_kitchen, r_dungeon_cell, r_server_room, r_banquet_room, r_nest, r_armory, r_mine, r_office, r_walk_in_closet, r_ball_room]
# set values
x = 0
y = 0
current_direction = 'n'
# while look for making rooms
# Random walk through grid of rooms, 
# any direction except backwards
while room_count < num_rooms:
    # randomly pick room type
    this_room_type = random.choice(room_type_list)
    # # inspection
    # print(this_room_type)
    # previous_room_direction = reverse_dirs[current_direction]
    possible_directions = ['n', 's', 'e' ,'w']
    # starts with a random choice:
    current_direction = random.choice(possible_directions)
    # save old values
    x_old = x
    y_old = y
    # save the location of the past room
    past_room = (x,y)
    # "don't go out of bounds"
    # check to make sure not out of bounds
    # while out of bounds, keep trying again for a not-out option
    while (x >= size_x - 1) or (y >= size_y - 1) or (x < 0) or (y < 0) or ((x,y) == past_room) or ((x,y) == (0,0)):
        # save old values
        x = x_old
        y = y_old
        # starts with a random choice:
        current_direction = random.choice(possible_directions)
        if current_direction == 'n':
            y += 1
        if current_direction == 's':
            y -= 1
        if current_direction == 'e':
            x += 1
        if current_direction == 'w':
            x -= 1
    # look for a previously made room (where you are looking 
    # to make a new room)
    # if not (already one there), make a new room
    if grid[y][x] is None:
        # # Create a room in the given direction
        # room = Room(room_count, this_room_type[0], this_room_type[1])
        #(title="Outside Cave Entrance", description="North of you, the cave mount beckons")
        room = Room(title = this_room_type[0], description = this_room_type[1], x, y)   
        # Save the room in the World grid
        grid[y][x] = room
        # update room counter, only if a new room is made
        room_count += 1
        # add room to room list:
        mask_dict = {'x':x, 'y':y}
        rooms_list.extend([mask_dict])
    # sets previous reverse room direction
    reverse_dirs = {"n": "s", "s": "n", "e": "w", "w": "e"}
    previous_room_direction = reverse_dirs[current_direction]
    # either way: Connect the new room to the previous room
    if previous_room is not None:
        # connect to the previous room...in DB
        previous_room.maker_connect_rooms(room, previous_room_direction)
        room.maker_connect_rooms(previous_room, current_direction)
        # doors list: add room to list:
        mask = [{'x': int(room.x), 'y': int(room.y)}, {'x': int(previous_room.x), 'y': int(previous_room.y)}]
        doors_list.extend([mask])
        # this saves a:  
        # will this work? pull up each tuple and slice that?
        mask3 = (room , previous_room)
        doors_list.extend(mask)
    # Update iteration variables
    previous_room = room
    # Saves the room-object for just the first room
    # to be used later as the starting location of
    # the player.
    if room_count == 1:
        first_room = room
    # # saves the room to django datatable
    # # or just makes everything break
    #room_obj_list.extend(room) 
    room.save()
print(rooms_list)
print(doors_list)

In [0]:
# create_world.py / this version only works in heroku
# this is the file that must be run, which dungeon making must be merged into
from django.contrib.auth.models import User
from adventure.models import Player, Room
from adventure.classes import Room_maker, World
# steps:
# step 1: make rooms
# step 2: make players
# erases pre-existing rooms
Room.objects.all().delete()
# # generate world:
# # set input values
w = World()
num_rooms = 100
width = 20
height = 20
w.generate_rooms(width, height, num_rooms)
players = Player.objects.all()
for p in players:
    p.currentRoom = w.first_room.id
    p.save()

In [0]:
Room_maker

In [0]:
# helper helperness!!!

from django.contrib.auth.models import User
from adventure.models import Player, Room
import random

Room.objects.all().delete()

population = ["crowded", "normal", "deserted"]

ambiance = ["bright", "dim", "normal", "grimy", "sterile"]

funct = ["card room", "hallway", "tunnel", "kitchen", "bar", "pool room"]

description_beginners = ["You find yourself in ",
                         "You have stumbled upon ", "You walk into "]

pop_desc = {
    "crowded": "Around you is a sea of bustling orcs and goblins.",
    "normal": "A few goblins are milling about.",
    "deserted": "The place looks deserted."
}

ambiance_desc = {
    "bright": "The glare of the torch light is almost too much for your eyes to bear.",
    "dim": "You can barely make out any shapes in the dim light.",
    "normal": "Things look fine here.",
    "grimy": "Everything has a fine layer of dust and filth upon it.",
    "sterile": "The walls and floor are spotless."
}

functional_desc = {
    "card room": "An orc holding a deck of cards looks up and smiles as you enter.",
    "hallway": "Just a hallway, not much to do but walk on.",
    "tunnel": "Water drips from the ceiling onto the muddy floor.",
    "kitchen": "An awful stench fills your nose.  Probably just roast halfling.",
    "bar": "The smell of stale grog makes your stomach turn.",
    "pool room": "The felt on the pool tables is faded and splattered with dark stains."
}


def room_generator(id):
    room_population = random.choice(population)
    room_ambiance = random.choice(ambiance)
    room_function = random.choice(funct)
    room_desc_begin = random.choice(description_beginners)
    title = f"{room_population.capitalize()} {room_function}"
    description = f"{room_desc_begin} a {room_population} {room_function}. {pop_desc[room_population]} {ambiance_desc[room_ambiance]} {functional_desc[room_function]}"
    return Room(id=id, title=title, description=description)

# input settings
num_room = 150
height = 20
width = 20

# list of room objects
rooms = []

for id in range(num_room):
    random_room = room_generator(id)
    rooms.append(random_room)

# for room in rooms:
#     room.save()

opposite = {
    "n": "s",
    "s": "n",
    "e": "w",
    "w": "e"
}

world = dict()


def popomatic():
    dir_int = random.randint(0, 3)
    dir = {
        0: "n",
        1: "s",
        2: "e",
        3: "w"
    }
    return dir[dir_int]

# takes in a direction and coordinate list (y, x), returns appropriate
# list representing 1 move in that direction


def move_coord(dir, coord_list):
    if dir == "n":
        return [coord_list[0] + 1, coord_list[1]]
    elif dir == "s":
        return [coord_list[0] - 1, coord_list[1]]
    elif dir == "e":
        return [coord_list[0], coord_list[1] + 1]
    else:
        return [coord_list[0], coord_list[1] - 1]


def hashable(list):
    hashableCoord = (list[0], list[1])
    return hashableCoord


# grab the first room
new_room = rooms.pop()
# mark this room as the origin point
origin = new_room
# find middle of nested list
current_coord = [height // 2, width // 2]
# record position of room in dict
world[hashable(current_coord)] = new_room
# set room coordinates
new_room.x = current_coord[1]
new_room.y = current_coord[0]
# save room to database
new_room.save()
# set current moving direction
cur_dir = popomatic()
stepper = random.randint(1, 5)
# move coordinates
# current_coord = move_coord(cur_dir, current_coord)
print(current_coord)
last_room = new_room

while len(rooms) > 0:
    # get the next room
    new_room = rooms.pop()
    # if stepper is <= 0, change direction, and reset stepper
    if stepper <= 0:
        stepper = random.randint(1, 3)
        cur_dir = popomatic()
    # while there is a room in cur_dir
    next_coord = move_coord(cur_dir, current_coord)
    # determine if within bounds, change direction if not
    while (0 > next_coord[0] or next_coord[0] > width) or (0 > next_coord[1] or next_coord[1] > height):
        cur_dir = popomatic()
        next_coord = move_coord(cur_dir, next_coord)
    while hashable(next_coord) in world:
        # if there is, move to it, connect the rooms
        last_room = world[hashable(current_coord)]
        curr_room = world[hashable(next_coord)]
        last_room.connectRooms(curr_room, cur_dir)
        curr_room.connectRooms(last_room, opposite[cur_dir])
        # save the room
        # last_room.save()
        # curr_room.save()
        # continue until there is no existing room at curr_coord
        current_coord = next_coord
        next_coord = move_coord(cur_dir, next_coord)
        last_room = curr_room
    # set new room in empty spot
    world[hashable(next_coord)] = new_room
    new_room.x = next_coord[1]
    new_room.y = next_coord[0]
    new_room.save()
    last_room.connectRooms(new_room, cur_dir)
    new_room.connectRooms(last_room, opposite[cur_dir])
    last_room = new_room
    curr_room = new_room
    current_coord = next_coord
    stepper -= 1


players = Player.objects.all()
for p in players:
    p.currentRoom = origin.id
    p.save()

print(world)

In [0]:
# helper helperness!!!
from django.contrib.auth.models import User
from adventure.models import Player, Room
import random
Room.objects.all().delete()
population = ["crowded", "normal", "deserted"]
ambiance = ["bright", "dim", "normal", "grimy", "sterile"]
funct = ["card room", "hallway", "tunnel", "kitchen", "bar", "pool room"]
description_beginners = ["You find yourself in ",
                         "You have stumbled upon ", "You walk into "]
pop_desc = {
    "crowded": "Around you is a sea of bustling orcs and goblins.",
    "normal": "A few goblins are milling about.",
    "deserted": "The place looks deserted."
}
ambiance_desc = {
    "bright": "The glare of the torch light is almost too much for your eyes to bear.",
    "dim": "You can barely make out any shapes in the dim light.",
    "normal": "Things look fine here.",
    "grimy": "Everything has a fine layer of dust and filth upon it.",
    "sterile": "The walls and floor are spotless."
}
functional_desc = {
    "card room": "An orc holding a deck of cards looks up and smiles as you enter.",
    "hallway": "Just a hallway, not much to do but walk on.",
    "tunnel": "Water drips from the ceiling onto the muddy floor.",
    "kitchen": "An awful stench fills your nose.  Probably just roast halfling.",
    "bar": "The smell of stale grog makes your stomach turn.",
    "pool room": "The felt on the pool tables is faded and splattered with dark stains."
}
def room_generator(id):
    room_population = random.choice(population)
    room_ambiance = random.choice(ambiance)
    room_function = random.choice(funct)
    room_desc_begin = random.choice(description_beginners)
    title = f"{room_population.capitalize()} {room_function}"
    description = f"{room_desc_begin} a {room_population} {room_function}. {pop_desc[room_population]} {ambiance_desc[room_ambiance]} {functional_desc[room_function]}"
    return Room(id=id, title=title, description=description)
# input settings
num_room = 150
height = 20
width = 20
# list of room objects
rooms = []
for id in range(num_room):
    random_room = room_generator(id)
    rooms.append(random_room)
# for room in rooms:
#     room.save()
opposite = {
    "n": "s",
    "s": "n",
    "e": "w",
    "w": "e"
}
world = dict()
def popomatic():
    dir_int = random.randint(0, 3)
    dir = {
        0: "n",
        1: "s",
        2: "e",
        3: "w"
    }
    return dir[dir_int]
# takes in a direction and coordinate list (y, x), returns appropriate
# list representing 1 move in that direction
def move_coord(dir, coord_list):
    if dir == "n":
        return [coord_list[0] + 1, coord_list[1]]
    elif dir == "s":
        return [coord_list[0] - 1, coord_list[1]]
    elif dir == "e":
        return [coord_list[0], coord_list[1] + 1]
    else:
        return [coord_list[0], coord_list[1] - 1]
def hashable(list):
    hashableCoord = (list[0], list[1])
    return hashableCoord
# grab the first room
new_room = rooms.pop()
# mark this room as the origin point
origin = new_room
# find middle of nested list
current_coord = [height // 2, width // 2]
# record position of room in dict
world[hashable(current_coord)] = new_room
# set room coordinates
new_room.x = current_coord[1]
new_room.y = current_coord[0]
# save room to database
new_room.save()
# set current moving direction
cur_dir = popomatic()
stepper = random.randint(1, 5)
# move coordinates
# current_coord = move_coord(cur_dir, current_coord)
print(current_coord)
last_room = new_room
while len(rooms) > 0:
    # get the next room
    new_room = rooms.pop()
    # if stepper is <= 0, change direction, and reset stepper
    if stepper <= 0:
        stepper = random.randint(1, 3)
        cur_dir = popomatic()
    # while there is a room in cur_dir
    next_coord = move_coord(cur_dir, current_coord)
    # determine if within bounds, change direction if not
    while (0 > next_coord[0] or next_coord[0] > width) or (0 > next_coord[1] or next_coord[1] > height):
        cur_dir = popomatic()
        next_coord = move_coord(cur_dir, next_coord)
    while hashable(next_coord) in world:
        # if there is, move to it, connect the rooms
        last_room = world[hashable(current_coord)]
        curr_room = world[hashable(next_coord)]
        last_room.connectRooms(curr_room, cur_dir)
        curr_room.connectRooms(last_room, opposite[cur_dir])
        # save the room
        # last_room.save()
        # curr_room.save()
        # continue until there is no existing room at curr_coord
        current_coord = next_coord
        next_coord = move_coord(cur_dir, next_coord)
        last_room = curr_room
    # set new room in empty spot
    world[hashable(next_coord)] = new_room
    new_room.x = next_coord[1]
    new_room.y = next_coord[0]
    new_room.save()
    last_room.connectRooms(new_room, cur_dir)
    new_room.connectRooms(last_room, opposite[cur_dir])
    last_room = new_room
    curr_room = new_room
    current_coord = next_coord
    stepper -= 1
players = Player.objects.all()
for p in players:
    p.currentRoom = origin.id
    p.save()
print(world)